# ACETONE tutorial #3
**Using the debug mode**

When developping new functionnalities (adding non-existents layers, changing/adding implementations, ...), it is quite common to do a first draft, try it, encounter somme bugs, bedug the code, try again, find other bugs, debug again, ... and so on. The fact that we need to debug the code means we need to find where the bugs occur first.

But, for ACETONE, using the framework as we are used to is not really helpful. Indeed, the framework's generated C code (and the python's inference model)  only returns the models output, leaving us no way of knowing whether the error occured in the first layers, or in the later ones. This behaviour has led to the framework's `debug_mode`, which we will use and explain in this notebook.

The first part is dedicated to generating the code, while the second part tackles the generation of a reference known to be true and the comparison with said reference.

* When running this notebook on Colab, we need to install ACETONE 
* If you run this notebook locally, run it in the environment in which you installed ACETONE

In [ ]:
# TODO Installs on collab

In [ ]:
# Cleaning the working environment
from pathlib import Path
from os import remove, listdir
files_directories = [Path("demo_squeezenet")]

for directory in files_directories:
    if directory.exists():
        for file in listdir(directory):
            remove(directory / file)

## Imports

In this notebook, we'll use as example the model `SqueezeNet 1.0` (with `opset-version==12`) given in [*ONNX's model zoo*](https://github.com/onnx/models?tab=readme-ov-file)

In [ ]:
import numpy as np
import numpy.random as rd

from acetone_nnet import CodeGenerator
from acetone_nnet import debug

In [ ]:
model_path = "../tests/models/squeezenet1/squeezenet1.onnx"
test_dataset = np.float32(rd.random((1,3,224,224)))
function_name = "demo_squeezenet"
nb_tests = 1

output_path = "demo_squeezenet"

## Generating the code

We first instantiate a `CodeGenerator` element with the debug parameter.


In [ ]:
# Debugging an onnx model
debug_mode = "onnx"

debug_generator = CodeGenerator(file=model_path,
                                test_dataset=test_dataset,
                                function_name=function_name,
                                nb_tests=nb_tests,
                                debug_mode=debug_mode)


debug_generator.generate_c_files(output_path)
outputs_python, targets_python = debug_generator.compute_inference(output_path)

Unlike in the "classic" mode, the  function `compute_inference` returns two elements. The first one, `outputs_python`, is a list regrouping the outputs of all the layers of interest, while the other one, `targets_python`, is a list containing the name and indice of the layer. Both lists are constructed such as `outputs_python[i]` is the output of the layer `targets_python[i]`.

In [ ]:
! make -C demo_squeezenet all
! ./demo_squeezenet/demo_squeezenet ./demo_squeezenet/output_c.txt

After compilating the code and running the newly created executable, another text file as been created : [debug_file.txt](./demo_squeezenet/debug_file.txt). This document contains both the name and indice of each layers (on odd ligns) and the ouput of those layers (on even ligns).

## Formatting ACETONE's outputs

After the parsing stage of ACETONE, a sorting algorithm is applied to the extracted list of layers, to ensure that they are weel ordered (no parent layer is after a child layer). This sorting stage allows us to work with the layers without worrying about wether all the inputs have been computed, or if we need to wait for another layer. But it has the inconvenience of changing the order of the layers from the original one in the model, thus requiring a sort on `outputs_python`.

In [ ]:
debug_file_path = output_path + "/debug_file.txt"

# Retrieving C's ouptut
outputs_c, targets_c = debug.extract_outputs_c(path_to_output=debug_file_path,
                                               data_type=debug_generator.data_type,
                                               nb_targets=len(debug_generator.debug_target))
# Ordering python's output
outputs_python, targets_python = debug.reorder_outputs(outputs_python, targets_python)



## Generating a reference

Once ACETONE's ouptut have been computed and formatted, we need a base reference to check if and when an error occured during the inference. 

In [ ]:
to_save = True
saving_path = output_path + "/debug_squeezenet.onnx"
otpimize_inputs = True

model, _, outputs_onnx = debug.debug_onnx(target_model=model_path,
                                          dataset=test_dataset,
                                          otpimize_inputs=otpimize_inputs,
                                          to_save=to_save,
                                          path=saving_path)


The `debug_onnx` function takes the model, modifies it for our problem (by adding ouptuts after the layers of interest), then runs the inference using the given dataset. 

## Comparing the outputs

We now can use our reference to check the framework's outputs, and locate, if they exists, errors in the implementation.

In [ ]:
# Comparing the result python with the result onnx
same = debug.compare_result(acetone_result=outputs_python,
                            reference_result=outputs_onnx,
                            targets=targets_python,
                            verbose=True)


In [ ]:
# Comparing the result c with the result onnx
same = debug.compare_result(acetone_result=outputs_c,
                            reference_result=outputs_onnx,
                            targets=targets_python,
                            verbose=True)

In [ ]:
# Comparing the result python with the result c
same = debug.compare_result(acetone_result=outputs_c,
                            reference_result=outputs_python,
                            targets=targets_python,
                            verbose=True)